In [2]:
import sqlite3
import csv

In [3]:
with open('./daum_kospi.csv', 'r', encoding='utf-8') as f:
    filereader = csv.reader(f)
    for idx, content in enumerate(filereader):
        if(idx < 5):
            print(content)
        else: break

['\ufeffdate', 'tradePrice', 'change', 'changePrice', 'accTradeVolume', 'accTradePrice', 'individualStraightPurchasePrice', 'foreignStraightPurchasePrice', 'institutionStraightPurchasePrice']
['2019-07-03 00:00:00', '2113.28', 'FALL', '8.74', '180753', '1415097', '25435352963', '-51579856319', '28986343951']
['2019-07-02 00:00:00', '2122.02', 'FALL', '7.72', '460830', '4107541', '24006908061', '34321459071', '-51500525649']
['2019-07-01 00:00:00', '2129.74', 'FALL', '0.88', '412291', '4730192', '-19121687014', '154486503785', '6138187770']
['2019-06-28 00:00:00', '2130.62', 'FALL', '3.7', '537274', '4282404', '-199483658718', '312263760802', '-109684798220']


In [6]:
# 메모리에 DB 생성
con = sqlite3.connect(':memory:')

In [7]:
query = """
CREATE TABLE kospi_table
(date VARCHAR(30) NOT NULL,
trade_price FLOAT NULL DEFAULT NULL,
change VARCHAR(30) NULL DEFAULT NULL,
changePrice FLOAT NULL DEFAULT NULL,
accTradeVolume BIGINT(20) NULL DEFAULT NULL,
accTradePrice BIGINT(20) NULL DEFAULT NULL,
individualStraightPurchasePrice BIGINT(20) NULL DEFAULT NULL,
foreignStraightPurchasePrice BIGINT(20) NULL DEFAULT NULL,
institutionStraightPurchasePrice BIGINT(20) NULL DEFAULT NULL,
PRIMARY KEY (date)
)
"""

In [8]:
# 실행
con.execute(query)

In [9]:
# 반영
con.commit()

In [10]:
# 데이터 삽입
statement = "INSERT INTO kospi_table VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)"

In [11]:
with open('./daum_kospi.csv', 'r', encoding='utf-8') as f:
    filereader = csv.reader(f)
    for idx, content in enumerate(filereader):
        con.execute(statement, content)

In [12]:
con.commit()

In [13]:
# 조회
cursor = con.execute('SELECT * FROM kospi_table')

In [14]:
rows = cursor.fetchall()

In [17]:
len(rows)

101

In [20]:
# WHERE 조건
cursor = con.execute('SELECT * FROM kospi_table WHERE date > "2019-07-01"')
rows = cursor.fetchall()
rows

[('2019-07-01 00:00:00',
  2129.74,
  'FALL',
  0.88,
  412291,
  4730192,
  -19121687014,
  154486503785,
  6138187770),
 ('2019-07-02 00:00:00',
  2122.02,
  'FALL',
  7.72,
  460830,
  4107541,
  24006908061,
  34321459071,
  -51500525649),
 ('2019-07-03 00:00:00',
  2113.28,
  'FALL',
  8.74,
  180753,
  1415097,
  25435352963,
  -51579856319,
  28986343951),
 ('\ufeffdate',
  'tradePrice',
  'change',
  'changePrice',
  'accTradeVolume',
  'accTradePrice',
  'individualStraightPurchasePrice',
  'foreignStraightPurchasePrice',
  'institutionStraightPurchasePrice')]

In [26]:
import pandas as pd
pd.DataFrame(rows[:-1], columns=rows[-1])

,﻿date,tradePrice,change,changePrice,accTradeVolume,accTradePrice,individualStraightPurchasePrice,foreignStraightPurchasePrice,institutionStraightPurchasePrice
0,2019-07-01 00:00:00,2129.74,FALL,0.88,412291,4730192,-19121687014,154486503785,6138187770
1,2019-07-02 00:00:00,2122.02,FALL,7.72,460830,4107541,24006908061,34321459071,-51500525649
2,2019-07-03 00:00:00,2113.28,FALL,8.74,180753,1415097,25435352963,-51579856319,28986343951


In [27]:
import json
import requests

In [28]:
con = sqlite3.connect('mystock.db')

In [29]:
url = 'https://finance.naver.com/item/sise_day.nhn?code=195440&page={}'

In [30]:
pd.read_html(requests.get(url.format(1)).text)[0].dropna()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.01.23,4615.0,80.0,4695.0,4710.0,4590.0,49820.0
2,2020.01.22,4695.0,145.0,4780.0,4800.0,4645.0,81399.0
3,2020.01.21,4840.0,75.0,4915.0,4965.0,4700.0,54413.0
4,2020.01.20,4915.0,135.0,4955.0,5090.0,4910.0,66824.0
5,2020.01.17,5050.0,10.0,5060.0,5100.0,4965.0,52772.0
9,2020.01.16,5060.0,30.0,5070.0,5150.0,4985.0,61506.0
10,2020.01.15,5090.0,215.0,4875.0,5090.0,4830.0,114396.0
11,2020.01.14,4875.0,125.0,5090.0,5090.0,4860.0,91123.0
12,2020.01.13,5000.0,145.0,4800.0,5270.0,4800.0,221862.0
13,2020.01.10,4855.0,150.0,4705.0,4930.0,4685.0,126381.0


In [31]:
tmp = pd.read_html(requests.get(url.format(1)).text)[0].dropna()

In [32]:
tmp.to_sql('stock_day', con, if_exists='append', index=False)

In [34]:
pd.read_sql_query('SELECT * FROM stock_day', con, parse_dates=['날짜'], index_col=['날짜'])

,종가,전일비,시가,고가,저가,거래량
날짜,,,,,,
2020-01-23,4615.0,80.0,4695.0,4710.0,4590.0,49820.0
2020-01-22,4695.0,145.0,4780.0,4800.0,4645.0,81399.0
2020-01-21,4840.0,75.0,4915.0,4965.0,4700.0,54413.0
2020-01-20,4915.0,135.0,4955.0,5090.0,4910.0,66824.0
2020-01-17,5050.0,10.0,5060.0,5100.0,4965.0,52772.0
2020-01-16,5060.0,30.0,5070.0,5150.0,4985.0,61506.0
2020-01-15,5090.0,215.0,4875.0,5090.0,4830.0,114396.0
2020-01-14,4875.0,125.0,5090.0,5090.0,4860.0,91123.0
2020-01-13,5000.0,145.0,4800.0,5270.0,4800.0,221862.0
